In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

from langchain.chains import LLMChain

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
)

In [2]:
# 1. LLM chain
# off-the-shelf chain -> 일반적인 목적을 가진 chain
# langchain에 아주 많고 유용함

# 실제로 무언가를 만들어볼때
# off-the-shelf chain보다는 우리가 직접 커스텀해서 만든 chain을 활용하기를 선호할 거
# off-the-shelf chain은 빠르게 시작할 수 있다는 점은 좋지만
# 프레임워크를 다루느라 머리싸매거나 off-the-shelf chain을 커스텀하기보다 직접 만드는 게 더 쉬울 수 있음 (langchain expression 언어를 활용해서)

In [3]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,
)

In [4]:
chain.predict(
    question="My name is Nico"
)  # 'Nice to meet you, Nico! Is there anything I can help you with?'



> Entering new LLMChain chain...
Prompt after formatting:
My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [5]:
chain.predict(question="I live in Seoul")  # 겁나 길게 서울 설명함



> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


', the capital city of South Korea. It is a vibrant and bustling metropolis with a rich history and culture. There are so many things to see and do in Seoul, from visiting ancient palaces and temples to exploring modern shopping districts and enjoying delicious Korean cuisine. I love living in Seoul because of its dynamic energy and the endless opportunities for new experiences. Plus, the public transportation system is excellent, making it easy to get around the city. Overall, Seoul is a fantastic place to call home.'

In [6]:
chain.predict(
    question="What is my name?"
)  # "I'm sorry, I don't know your name. Can you please tell me?" -> 디버깅 해보자



> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, I don't know your name. Can you please tell me?"

In [7]:
# LLMChain에 verbose=True로 디버깅함
# 대화 내역이 프롬프트에 계속 추가되지 않는다는 점 발견

# 그럼 대화 내역을 넘겨줘야지

In [8]:
memory.load_memory_variables({})  # 대화내역은 잘 작동하고 있음 -> 요약도

{'history': "System: The human introduces themselves as Nico and the AI responds by greeting them and asking how it can assist them. Nico shares that they live in Seoul, the capital city of South Korea, and describes it as a vibrant and bustling metropolis with a rich history and culture. They love living in Seoul because of its dynamic energy, endless opportunities for new experiences, and excellent public transportation system.\nHuman: What is my name?\nAI: I'm sorry, I don't know your name. Can you please tell me?"}

In [9]:
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

# AI가 대화내역을 기억하여 위의 question에 답할 수 있게끔 하고 싶음

In [13]:
# 대화내역과 memory는 human과 LLM의 입력, 출력을 기반으로 업데이트되고 있음
# 이제 Conversation Memory Class에게 template 안에 내용을 넣으라고 해줘야 함

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",  # template 안에 memory가 대화내역을 저장하도록 한 곳을 적는거
)

# 그럼 memory는 template 안에 대화기록을 저장하는 공간을 찾을 거고
# 그 메모리의 기록(대화내역)을 template에 넣을 거임

In [14]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    # template에 맞춰서 메모리를 넣기위해서
    verbose=True,
)

# memory -> chain 순서 대로 안하면 key를 못찾음

In [15]:
chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Nico
    You:


> Finished chain.


'Hello Nico! How can I assist you today?'

In [16]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


"That's great, Seoul is a vibrant and exciting city. Is there anything specific you need assistance with related to living in Seoul?"

In [17]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great, Seoul is a vibrant and exciting city. Is there anything specific you need assistance with related to living in Seoul?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Nico.'

In [18]:
# 잘 작동함
# memory의 max_token_limit를 조절하고 대화내역이 많이 쌓이면 과거 내역이 요약되는 걸 볼 수 있음